Logistic Regression to predict if songs contain love

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
import seaborn as sns


from sklearn.linear_model import LogisticRegression

In [ ]:
#Read in datasets
spotify_data = pd.read_csv("../data/Hot 100 Audio Features.csv")
billboard_data = pd.read_csv("../data/Hot Stuff.csv")

In [ ]:
#Cleaning from EDA

def point_calculation_final(val): 
    new_val = 101-val
    return new_val
billboard_data["Popularity Points Awarded"] = billboard_data["Week Position"].apply(point_calculation_final)

total_points_by_song = pd.DataFrame(billboard_data.groupby(billboard_data["Song"])["Popularity Points Awarded"].sum())

super_popular_pop = total_points_by_song.sort_values(by="Popularity Points Awarded", ascending=False)
list_of_songs = spotify_data["Song"].to_list()

points_by_song_real = pd.DataFrame(billboard_data.groupby(["Song", "Performer"])["Popularity Points Awarded"].sum().reset_index())

real_super_popular_pop = points_by_song_real.sort_values(by="Popularity Points Awarded", ascending=False)

spotify_data["Popularity Points Awarded"] = ''
points_songs_list = points_by_song_real["Popularity Points Awarded"].to_list()
artists_songs_list = points_by_song_real["Performer"].to_list()
titles_songs_list = points_by_song_real["Song"].to_list()
def append_pts_test(points, artists, titles, spotify):
    for i in range(len(points)): 
        point_value = points[i]
        artist = artists[i]
        title = titles[i]
        index = spotify[(spotify['Performer'] == artist) & (spotify['Song'] == title)].index
        spotify.loc[index, "Popularity Points Awarded"] = point_value
    return spotify

spotify_data = spotify_data.merge(points_by_song_real, on=['Performer', 'Song'], how='left') 

spotify_data = spotify_data.drop(["Popularity Points Awarded_x"], axis=1)
spotify_data["Popularity Points Awarded"] = spotify_data["Popularity Points Awarded_y"]
spotify_data = spotify_data.drop(["Popularity Points Awarded_y"], axis=1)
song_attribute_data = spotify_data

titles = song_attribute_data["Song"]
titles = titles.str.split(pat=" ", expand=True)

song_attribute_data["Has Word Love?"] = song_attribute_data['Song'].str.contains("Love")
song_attribute_data_love = song_attribute_data[song_attribute_data["Has Word Love?"]]


In [ ]:
#Prepare X and y
y = [song_attribute_data["Has Word Love?"]] 
X = song_attribute_data.loc[:,["speechiness", "valence", "tempo", "time_signature", "Popularity Points Awarded"]]

def minmax(z):
    z = (z-min(z))/(max(z)-min(z))
    return z
X = X.apply(minmax,axis=1)

In [ ]:
#Run Regression 
reg = LogisticRegression(penalty = None,
                         fit_intercept=True, 
                         solver = 'newton-cholesky', 
                         max_iter=1000).fit(X,y)
print('Accuracy: ', reg.score(X,y))

